## Import and create processing functions

In [ ]:
# make sure to install these packages before running:
# pip install sodapy
# pip install addfips
# pip install opencage
import numpy as np
import pandas as pd
import addfips
from sodapy import Socrata
from opencage.geocoder import OpenCageGeocode

# Load functions for process and create prevalence of chronic disease compiled on age groups at each county
# have questions, please send your inquiry to ashkan.farahani@gmail.com

# couties for District of colombia and also "Guam" is diffrent. A function is create to process the outputs 
# from Opencage(Geolocation API) regarding this matter

def city_to_county(x):
    if x.get("state") != 'Guam':
        if x.get("county"):
            d = x.get("county")
            # Note that subsetting dictionary with dict['county'] produced error. instead use .get method to retrive county names
        else:
            # for DC county is None
            d = x.get("city")
    else:
        d = "Guam"
    return d
    
# pd.pivot method has a bug and unable to do multiIndexing. Here is the solution. 
def multiIndex_pivot(df, index = None, columns = None, values = None):
    # https://github.com/pandas-dev/pandas/issues/23955
    output_df = df.copy(deep = True)
    if index is None:
        names = list(output_df.index.names)
        output_df = output_df.reset_index()
    else:
        names = index
    output_df = output_df.assign(tuples_index = [tuple(i) for i in output_df[names].values])
    if isinstance(columns, list):
        output_df = output_df.assign(tuples_columns = [tuple(i) for i in output_df[columns].values])  # hashable
        output_df = output_df.pivot(index = 'tuples_index', columns = 'tuples_columns', values = values) 
        output_df.columns = pd.MultiIndex.from_tuples(output_df.columns, names = columns)  # reduced
    else:
        output_df = output_df.pivot(index = 'tuples_index', columns = columns, values = values)    
    output_df.index = pd.MultiIndex.from_tuples(output_df.index, names = names)
    return output_df

# When there is very small proportion of response to a certain question measuring some condition in BRFSS survey, the answer 
# is left NA. 
# The NA can be obtained by fetching at the opposite answer for the same question in the same population of interetst (same FIPS and age group) 
# and then impute NA by (100 - opposite response) 
def fill_na_prevalence(df, target_responseid, opposite_responseid,colname):
    convert_dict = {'data_value': float}
    df = df.astype(convert_dict) 
    df_pivot = df.pipe(multiIndex_pivot, index = ["fips","break_out"], columns = 'responseid', values = 'data_value')
    index_na = df_pivot[target_responseid].index[df_pivot[target_responseid].isna()]
    df_pivot.loc[index_na,target_responseid] = 100 - df_pivot.loc[index_na,opposite_responseid]
    df_final = df_pivot.drop(opposite_responseid,axis=1)
    df_final.columns = colname
    return df_final


# Naiive method for the same job
def replace_na(df,response):
    
    convert_dict = {'data_value': float}
    df = df.astype(convert_dict) 
    df = df.sort_values(['fips','break_out'])
    # extract fips and age group of those with null values in prevelance
    fips_age_null = df[(df.data_value.isnull()) & (df.responseid == response)][['fips','break_out']]
    # replace with 1 - prevalence(opposite non_NA)
    for i in range(fips_age_null.shape[0]):
        fip = fips_age_null.iloc[i,0]
        age = fips_age_null.iloc[i,1]
        df.loc[fips_age_null.index[i],"data_value"] = 100 - df[(df.data_value.notnull()) & (df.fips == fip) & (df.break_out == age )]['data_value'].values
    df1 = df[df.responseid == response]
    return df1

# Create a Prevalence extractor function
def prevalence(df,colname=None,target_responseid= None, opposite_responseid= None, fips = None, more_than_two_resp = False, key='1b9eb42a522c4c37a3be8ec886aa1494'):
    ## First pull county from geolocation from OpenCage package 
    # covert geolocation to location by state, county or city
    # Key is created and obtained by signing up at https://opencagedata.com/users/sign_up
    # Limit is 2500 requestes a day: each row of the geolocation counts as one request
    # key = '1b9eb42a522c4c37a3be8ec886aa1494' #other
    # key = '92a49e3bcb214d519bb4c0fa5fd9bca0' #ashkan
    
    # description of input parameters
    # df: dataframe to pull from 
    # colname: is the name of chronic disease dataframe (e.g. obesity)
    # target_responseid: is the response id of interest for instance RESP046 is a 'yes' response to a question: "if you ever had diabetes?"
    # opposite_responseid: is the opposite response of target for instance RESP054 is a 'no' response to a question: "if you ever had diabetes?"
    # fips: only if a previous list of counties with corresponding FIPS codes are available. the format should be the multiIndex of (latitude,logitude) with values of FIPS and county
    # more_than_two_resp: only True situation when there is more than two responses are available. the whole output will change. require post processing
    # key is the created key at https://opencagedata.com/users/sign_up. if the error message says, your key has reached its limit for today, you should change your key
    
    geocoder = OpenCageGeocode(key)
    df['latitude']  = df.apply(lambda x: float(x['geolocation']['latitude']), axis=1)
    df['longitude'] = df.apply(lambda x: float(x['geolocation']['longitude']), axis=1)
    df = df.assign(geo = [tuple(i) for i in df[['latitude','longitude']].values])
    
    
    # Since the limit is 2500 per day, after the first pull, the "reverse_geo_fips" function uses the previous geo pulls to avoid current redundant geo pulls.
    def reverse_geo_fips(geo_tuple,fips):
        if fips is not None:
            if geo_tuple in fips.index:
                fips_county = np.unique(fips.loc[geo_tuple,'county'])[0]
            else:
                fips_components = geocoder.reverse_geocode(geo_tuple[0], geo_tuple[1])[0]['components']
                fips_county     = str(city_to_county(fips_components)).replace('County','')
                # drop 'County' at the end of county names
        else:
            fips_components = geocoder.reverse_geocode(geo_tuple[0], geo_tuple[1])[0]['components']
            fips_county     = str(city_to_county(fips_components)).replace('County','')
            
        return fips_county
                
    df['county'] = df.apply(lambda x: reverse_geo_fips(x['geo'],fips), axis=1)       
    
    # Adding county FIPS
    af = addfips.AddFIPS()
    df['fips'] = df.apply(lambda x: af.get_county_fips(x['county'].strip(), x['locationabbr'].strip()),axis=1)
    df = df.sort_values(['fips','break_out'])
    # create a FIPS dataframe for future use. other risk factors(e.g. kidney) might have been pulled from the same location and no need to use geolocation API
    fips_df = df[['latitude', 'longitude', 'county', 'fips']]
    fips_df = fips_df.drop_duplicates(subset=['latitude', 'longitude'])
    fips_df = fips_df.assign(tuple_index = [tuple(i) for i in fips_df[['latitude', 'longitude']].values])
    fips_df.index = pd.MultiIndex.from_tuples(fips_df['tuple_index'],names=['latitude', 'longitude'])
    fips_df = fips_df.drop(['latitude','longitude','tuple_index'],axis=1)
    
    if more_than_two_resp:
        df_notnull = df.copy()
        
    else:
        
        if df['data_value'].isnull().sum() > 0:
            df_notnull = fill_na_prevalence(df, target_responseid, opposite_responseid, colname)
        else:
            df_notnull = df[['fips','break_out','data_value']]
            df_notnull = df_notnull.assign(tuple_index = [tuple(i) for i in df_notnull[['fips','break_out']].values])
            df_notnull.index = pd.MultiIndex.from_tuples(df_notnull['tuple_index'],names=['fips','break_out'])
            df_notnull =  df_notnull.drop(['fips','break_out','tuple_index'],axis=1)
            df_notnull.columns = colname
    
    return(df_notnull, fips_df)

In [2]:
# Data is pulled from CDC BRFSS API at Socrata - https://dev.socrata.com/foundry/chronicdata.cdc.gov/dttw-5yxu
# to check for future updates and new data check BRFSS home page: https://www.cdc.gov/brfss/
# To check BRFSS data portal: https://chronicdata.cdc.gov/browse?category=Behavioral+Risk+Factors
# To check updates on prevalence data: https://www.cdc.gov/brfss/data_tools.htm
# Create an account and app_token before using. Otherwise public use is limited
client = Socrata("chronicdata.cdc.gov", 
                 app_token="JSqven6RHvdAGCnxNIEfUC51z",
                 username="ashkan.aliabadifarahani@mavs.uta.edu", password="CC19LP_ashkan!")

# The pre-existing condition is compiled from CDC BRFSS based on published underlying condition by CDC here: https://www.cdc.gov/coronavirus/2019-ncov/need-extra-precautions/evidence-table.html
# The most Consistent Evidence ------------------------------------------------------------------------------------------------------------------------------------- 

## Cardiovascular - Stroke, Coronary Artery Diseaese # ResponseID = 'RESP203' those who have cardivascular disease
#myocardial_infarction   = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='CVDINFR4'")
#coronary                = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='CVDCRHD4'")
#stroke                  = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='CVDSTRK3'")
Cardiovascular           = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='_MICHD' AND Break_Out_Category ='Age Group'")
cvs_df                   = pd.DataFrame.from_records(Cardiovascular)
cvs_pv, fips_cvs         = prevalence(cvs_df,['Cardivascular'],'RESP203','RESP204')

## Cancer - skin and other - also will try data from CDC, National Cancer Institute (NCI)
skin                    = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='CHCSCNCR' AND Break_Out_Category ='Age Group'")
cancer_other            = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='CHCOCNCR' AND Break_Out_Category ='Age Group'")
              
## Kidney  - were you ever told you have kidney disease? ResponseID = 'RESP046':Yes
kidney                  = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='CHCKDNY1' AND Break_Out_Category ='Age Group' ")
kidney_df               = pd.DataFrame.from_records(kidney)
kidney_pv, fips_kidney  = prevalence(kidney_df,['Kidney'], 'RESP046', 'RESP054',fips=fips_cvs)

## Respiratory - Ever told you have chronic obstructive pulmonary disease, C.O.P.D., emphysema or chronic bronchitis?
COPD                    = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='CHCCOPD1' AND Break_Out_Category ='Age Group'")
copd_df                 = pd.DataFrame.from_records(COPD)
COPD_pv, fips_COPD      = prevalence(copd_df, ['COPD'], 'RESP046', 'RESP054',fips=fips_kidney)

## Obesity - Obesity (BMI> 30)
obesity                 = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='_BMI5CAT' AND ResponseID = 'RESP039' AND Break_Out_Category ='Age Group'")
obesity_df              = pd.DataFrame.from_records(obesity)
obesity_pv, fips_obesity= prevalence(obesity_df,['Obesity'],fips=fips_COPD)

## Diabetes - Ever told you have diabetes? 4 response yes,yes_pregnancy,no_borderline_diabetes, no : we add the first three
#diabetes_age            = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='DIABAGE2'")
#pre_diab                = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='PREDIAB1'")
diabetes                = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='DIABETE3' AND Break_Out_Category ='Age Group'")
diabetes_df             = pd.DataFrame.from_records(diabetes)
diabetes, fips_diabetes = prevalence(diabetes_df,fips=fips_obesity,more_than_two_resp=True)
diabetes_pivot          = diabetes.pipe(multiIndex_pivot, index = ["fips","break_out"], columns = 'responseid', values = 'data_value')
diabetes_pivot.fillna('0',inplace=True)
diabetes_pivot          = diabetes_pivot.astype({'RESP051':float, 'RESP046':float,'RESP052':float  })
diabetes_pivot["Diabetes"] = diabetes_pivot['RESP051'] + diabetes_pivot['RESP046'] + diabetes_pivot['RESP052']  
diabetes_pv             = diabetes_pivot["Diabetes"]


# Mixed Evidence ----------------------------------------------------------------------------------------------------------------------------------------------------
# Asthma - ever 
asthma                  = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='_LTASTH1' AND ResponseID = 'RESP046' AND Break_Out_Category ='Age Group'")
asthma_df               = pd.DataFrame.from_records(asthma)
asthma_pv, fips_asthma  = prevalence(asthma_df,['Asthma'],fips=fips_obesity)

# Asthma - current 
asthma_now              = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='_CASTHM1' AND Break_Out_Category ='Age Group'")
asthmanow_df            = pd.DataFrame.from_records(asthma_now)
asthmanow_pv,fips_asthma  = prevalence(asthmanow_df,['AsthmaNow'],'RESP046', 'RESP054',fips=fips_diabetes)

# Current Smoker
current_smoker          = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='_RFSMOK3' AND Break_Out_Category ='Age Group'")
current_smoker_df = pd.DataFrame.from_records(current_smoker)
current_smoker_pv, fips_smoker = prevalence(current_smoker_df,['Current_Smoker'],'RESP046', 'RESP054',fips=fips_asthma)

# Former Smoker
former_smoker           = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='_SMOKER3' AND ResponseID = 'RESP070' AND Break_Out_Category ='Age Group'")
former_smoker_df        = pd.DataFrame.from_records(former_smoker)
former_smoker, fips_smoker = prevalence(former_smoker_df, fips=fips_asthma, more_than_two_resp=True)
former_smoker_pivot = former_smoker.pipe(multiIndex_pivot, index = ["fips","break_out"], columns = 'responseid', values = 'data_value')
former_smoker_pivot.fillna('0',inplace=True)
former_smoker_pivot.columns = ['Former_Smoker']
former_smoker_pv = former_smoker_pivot.copy()


#asthma                  = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='ASTHMA3'")
#asthma_now              = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='ASTHNOW'")
#shortness_of_breath     = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='COPDBRTH'")
#breat_prob_diag         = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='COPDBTST'")
#tabaco_years            = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='COPDSMOK'")
#smoke                   = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='SMOKDAY2'")
#smoke_quit              = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='STOPSMK2'")
#E_cig                   = client.get_all("dttw-5yxu",where="Year=2018 AND QuestionID ='ECIGNOW'")

In [3]:
result = pd.concat([cvs_pv,kidney_pv,COPD_pv,obesity_pv,diabetes_pv,asthma_pv,current_smoker_pv,former_smoker_pv],axis=1)

In [4]:
result

Cardivascular  Kidney   COPD Obesity  Diabetes Asthma  \
fips  break_out                                                          
01021 18-24               0.00    1.45   3.95   19.27      0.00  24.31   
      25-34               2.50    1.69   3.94   38.07      4.37  16.95   
      35-44               5.84    1.32   4.59   41.95      6.94   15.6   
      45-54               6.22    4.05  11.95   39.83     13.10  16.03   
      55-64              16.08    5.68  16.81   43.67     26.63  15.53   
...                        ...     ...    ...     ...       ...    ...   
72073 25-34               4.14    2.16   3.14   35.88      2.19  20.82   
      35-44               6.04    1.95   3.85   37.78      8.02  18.04   
      45-54               9.28    3.47   8.77   40.42     15.69  18.53   
      55-64              14.77    5.42   6.77   37.93     28.85  19.51   
      65+                20.12    5.28   9.94   25.43     38.87  15.74   

                 Current_Smoker Former_Smoker  
fips  break_out                                
01021 18-24               13.25          8.49  
      25-34               24.71         18.65  
      35-44               25.08          20.9  
      45-54               23.12         19.12  
      55-64               21.30         32.34  
...                         ...           ...  
72073 25-34               13.16         11.27  
      35-44               11.45         15.22  
      45-54               11.90         16.66  
      55-64               11.49         24.84  
      65+                  5.54         28.32  

[318 rows x 8 columns]